In [7]:
%matplotlib inline
import os
import numpy as np
import xarray as xr
from datetime import datetime

from itertools import product

import matplotlib.pyplot as plt

import pop_tools
import util

Generate grid file for ETOPO1 dataset.

In [8]:
%%time
scrip_grid_file = util.file_name_grid('etopo1')

if not os.path.exists(scrip_grid_file): 
    
    # check that this conforms to the grid of the original dataset
    file_src_data = util.datasets['etopo1']['urlpath']
    ds = xr.open_dataset(file_src_data)
    grid_imask = xr.where(ds.z < 0, 1., 0.).values.astype(np.int32)
    plt.pcolormesh(grid_imask[::10, ::10])
    plt.colorbar()
    
    # get/write scrip grid
    dso = util.latlon_to_scrip(
        nx=21600, 
        ny=10800, 
        lon0=-180., 
        file_out=scrip_grid_file
    )

    # extract grid coordinates to make check below
    dims_grid = tuple(dso.grid_dims.values[::-1])
    ny, nx = dims_grid
    lat = dso.grid_center_lat.values.reshape(dims_grid)[:, 0]
    lon = dso.grid_center_lon.values.reshape(dims_grid)[0, :]

    np.testing.assert_allclose(lat, ds.y.values)
    np.testing.assert_allclose(lon, ds.x.values)

    dso.info()

CPU times: user 49 µs, sys: 40 µs, total: 89 µs
Wall time: 95.1 µs


## Generate POP grid files

In [9]:
for grid_name in util.POP_grids:
    scrip_grid_file = util.file_name_grid(grid_name)

    if not os.path.exists(scrip_grid_file):            
        print(f'generating {scrip_grid_file}')
        
        grid = pop_tools.get_grid(grid_name)
        del grid.attrs['region_mask_regions']
        grid.to_netcdf(scrip_grid_file)



Generate remapping files

In [10]:
#%%bash
# qsub not available from Casper
#qsub < esmf_gen_weights_etopo1_to_POP.pbs

In [11]:
#%%bash
# qsub not available from Casper
#qsub < esmf_gen_weights_POP_to_POP.pbs